In [33]:
seed_url = 'https://www.youtube.com/watch?v=din4SiHB-nc'
n_suggested_videos = 50
file_name = 'catdog_50_morten' # Don't type any file ending, it will automatically save in .json and .csv

In [34]:
# SETUP IMPORTS

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pandas as pd
import datetime
import random
import time

In [35]:
# SETUP DRIVER

profile = webdriver.FirefoxProfile('C:/Users/hanse/AppData/Roaming/Mozilla/Firefox/Profiles/mg75i8t2.default-release')

profile.set_preference("dom.webdriver.enabled", False)
profile.set_preference('useAutomationExtension', False)
profile.update_preferences()
desired = DesiredCapabilities.FIREFOX

driver = webdriver.Firefox(executable_path=r'geckodriver.exe', firefox_profile=profile,
                           desired_capabilities=desired)

In [36]:
# DEFINE HELPER FUCNTIONS
def get_duration_and_url(suggested_video):
    url = suggested_video.find_element_by_id('thumbnail').get_attribute('href')
    duration = 0
    try:
        duration = suggested_video.find_element_by_class_name('style-scope ytd-thumbnail-overlay-time-status-renderer').text
        if(duration.count(':') == 0):
            duration = int(duration)
        elif(duration.count(':') == 1):
            duration = duration.split(':')
            duration = (int(duration[0])*60) + int(duration[1])
        elif (duration.count(':') == 2):
            duration = duration.split(':')
            duration = (int(duration[0])*60*60) + (int(duration[1])*60) + int(duration[2])
    except:
        print("Duration not found for:", url)
    return (duration, url)

def scroll_down(n):
    y_position = 0
    step_size = 100
    start = time.time()
    elapsed_time = 0    
    
    while len(driver.find_elements_by_tag_name('ytd-compact-video-renderer')) < n and elapsed_time < 60:
        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        time.sleep(random.uniform(0.05,0.1))

        # If the browser has skipped past the infinity loader for videos, we go slowly from the top and down
        elapsed_time = time.time() - start
        if elapsed_time > 10:
            print('Time to scroll slowly')

            slow_start = time.time()
            slow_elapsed_time = 0
            driver.execute_script("window.scrollTo(0, "+str(y_position)+")")
            while len(driver.find_elements_by_tag_name('ytd-compact-video-renderer')) < n and slow_elapsed_time < 30:
                y_position += step_size
                time.sleep(0.5)
                driver.execute_script("window.scrollTo(0, "+str(y_position)+")")
                slow_elapsed_time = time.time() - slow_start
    driver.execute_script("window.scrollTo(0, 800)")
    
def get_video_info(id, parent_id, url, duration):
    driver.get(url)
    time.sleep(random.uniform(0.5,1))

    scroll_down(n_suggested_videos)
    time.sleep(random.uniform(1.5,2))

    title = pd.NA
    views = pd.NA
    likes = pd.NA
    date = pd.NA
    channel = pd.NA
    subscribers = pd.NA
    subscribed = pd.NA
    comments = pd.NA

    try:
        if driver.find_element_by_class_name('title.style-scope.ytd-video-primary-info-renderer'):
            title = driver.find_element_by_class_name('title.style-scope.ytd-video-primary-info-renderer').text
        
        if driver.find_element_by_class_name('view-count.style-scope.ytd-video-view-count-renderer'):
            views = int(driver.find_element_by_class_name('view-count.style-scope.ytd-video-view-count-renderer').text.split(' ')[0].replace(',', ''))
        
        likes_wrapper = driver.find_elements_by_class_name('style-scope.ytd-toggle-button-renderer.style-text')[1]
        if likes_wrapper.get_attribute("aria-label") is not None:
            likes = likes_wrapper.get_attribute("aria-label").split(' ')[0].replace(',', '')
        else:
            likes = likes_wrapper.text.split('.')
            if(len(likes) == 2):
                leading = likes[0]
                latter = likes[1].split(' ')[0]
                latter = latter[0] + latter[-1].replace('K', '00').replace('M', '00000').replace('B', '00000000')
                likes = int(leading + latter)
            else:
                likes = int(likes[0].replace('K', '000').replace('M', '000000').replace('B', '000000000').split(' ')[0])
        likes = int(likes)

        if driver.find_element_by_class_name('style-scope.ytd-channel-name'):
            channel = driver.find_element_by_class_name('style-scope.ytd-channel-name').text

        if driver.find_element_by_id('owner-sub-count'):
            subscribers = driver.find_element_by_id('owner-sub-count').text.split('.')
            if(len(subscribers) == 2):
                leading = subscribers[0]
                latter = driver.find_element_by_id('owner-sub-count').text.split('.')[1].split(' ')[0]
                latter = latter[0] + latter[-1].replace('K', '00').replace('M', '00000').replace('B', '00000000')
                subscribers = int(leading + latter)
            else:
                subscribers = int(subscribers[0].replace('K', '000').replace('M', '000000').replace('B', '000000000').split(' ')[0])
        
        if driver.find_element_by_class_name('style-scope.ytd-subscribe-button-renderer'):
            subscribed = True if driver.find_element_by_class_name('style-scope.ytd-subscribe-button-renderer').get_attribute("aria-label").split(' ')[0] == 'Unsubscribe' else False

        if driver.find_elements_by_class_name('style-scope.ytd-video-primary-info-renderer'):
            date = driver.find_elements_by_class_name('style-scope.ytd-video-primary-info-renderer')[12].text.replace('Premiered ', '').replace('Started streaming on ', '').replace('Streamed live on ', '')
            if(date != ''):
                format_str = '%b %d, %Y' # The format
                date = datetime.datetime.strptime(date, format_str).strftime('%d/%m/%Y')

        if driver.find_element_by_class_name('count-text.style-scope.ytd-comments-header-renderer'):
            comments = int(driver.find_element_by_class_name('count-text.style-scope.ytd-comments-header-renderer').text.split(' ')[0].replace(',', ''))

        return {'id': id, 'parent_id': parent_id, 'channel': channel, 'subscribers': subscribers, 'subscribed': subscribed, 'title': title, 'views': views, 'likes': likes, 'comments': comments, 'duration': duration, 'date': date, 'link': url}
    except Exception as e:
        print('Error while getting video info, msg:', e)
        return {'id': id, 'parent_id': parent_id, 'channel': channel, 'subscribers': subscribers, 'subscribed': subscribed, 'title': title, 'views': views, 'likes': likes, 'comments': comments, 'duration': duration, 'date': date, 'link': url}

In [37]:
# SCRAPER
# Locating elements: https://selenium-python.readthedocs.io/locating-elements.html

unique_id_count = 1
print('Starting scraper with seed url', seed_url)

videos = []
video = get_video_info(unique_id_count, 0, seed_url, 60)
videos.append(video)
root_id = unique_id_count

time.sleep(random.uniform(1,2))
suggested_videos = [get_duration_and_url(v) for v in driver.find_elements_by_tag_name('ytd-compact-video-renderer')[:n_suggested_videos]]
print('root id:', root_id, "got children:", suggested_videos)
for i, (duration, s_vid_url) in enumerate(suggested_videos):
    print('---------------------------------------------------------------------')
    unique_id_count += 1
    
    parent_id = unique_id_count
    video = get_video_info(unique_id_count, root_id, s_vid_url, duration)
    print('Appending parent no.:', str(i+1)+'/'+str(n_suggested_videos), '| id:', parent_id, "| info:", video)
    videos.append(video)

    time.sleep(random.uniform(1,2))
    nested_suggested_videos = [get_duration_and_url(v) for v in driver.find_elements_by_tag_name('ytd-compact-video-renderer')[:n_suggested_videos]]
    print('Getting children:', nested_suggested_videos)
    for nested_i, (nested_duration, nested_s_vid_url) in enumerate(nested_suggested_videos):
        unique_id_count += 1

        nested_id = unique_id_count
        nested_video = get_video_info(nested_id, parent_id, nested_s_vid_url, nested_duration)
        print('Appending child for parent no.:', str(i+1)+'/'+str(n_suggested_videos), '| id:', parent_id, "| child no.:", str(nested_i+1)+'/'+str(n_suggested_videos), "| child id:", nested_id, "| info:", nested_video)
        videos.append(nested_video)
    
    print('Subgoal save parent no.', str(i+1)+'/'+str(n_suggested_videos))
    df = pd.DataFrame(videos)
    df.to_csv(file_name + '.csv')
    df.to_json(file_name + '.json')

df = pd.DataFrame(videos)
df.to_csv(file_name + '.csv')
df.to_json(file_name + '.json')

df

Starting scraper with seed url https://www.youtube.com/watch?v=din4SiHB-nc
Duration not found for: https://www.youtube.com/watch?v=okXj0s1oZgg
Duration not found for: https://www.youtube.com/watch?v=1znjHDiqBk8
Duration not found for: https://www.youtube.com/watch?v=rsCsA_RmAjE
Duration not found for: https://www.youtube.com/watch?v=9yjZpBq1XBE
Duration not found for: https://www.youtube.com/watch?v=5k5gVe4H-A0
Duration not found for: https://www.youtube.com/watch?v=094y1Z2wpJg
Duration not found for: https://www.youtube.com/watch?v=5qap5aO4i9A
Duration not found for: https://www.youtube.com/watch?v=C-nALYF73hU
Duration not found for: https://www.youtube.com/watch?v=3RkhZgRNC1k
Duration not found for: https://www.youtube.com/watch?v=WJqx-ZRtP7c
Duration not found for: https://www.youtube.com/watch?v=85pLB5w4quQ
Duration not found for: https://www.youtube.com/watch?v=BkD2nN5275c
Duration not found for: https://www.youtube.com/watch?v=j5v8D-alAKE
Duration not found for: https://www.youtu

,id,parent_id,channel,subscribers,subscribed,title,views,likes,comments,duration,date,link
0,1,0,Funny Animals' Club,180000,False,The Best Funny Videos Of Cats And Dogs 🐶😹 - Fu...,23791411,89883,2200,60,18/09/2021,https://www.youtube.com/watch?v=din4SiHB-nc
1,2,1,Rufus,115000,False,THE BEST CUTE AND FUNNY ANIMAL VIDEOS OF 2020!,7071807,4387,88,,27/12/2020,https://www.youtube.com/watch?v=okXj0s1oZgg
2,3,2,Rufus,115000,False,THE BEST CUTE AND FUNNY CAT VIDEOS OF 2020! 🐱,2532177,2343,86,3628,03/01/2021,https://www.youtube.com/watch?v=fQ_DTui804I
3,4,2,The Timer,27800,False,Pomodoro Technique 4 x 25 min - Study Timer 2h,2260186,40344,909,7204,24/03/2017,https://www.youtube.com/watch?v=1znjHDiqBk8
4,5,2,Rain Sleep,[],<NA>,Deje que el insomnio se convierta en pasado co...,214,32,<NA>,0,<NA>,https://www.youtube.com/watch?v=Z3pa9aVqO8k
...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2538,2492,Summoning Salt,1200000,False,Super Mario Bros: The Human Limit,3135700,156231,10995,,30/05/2021,https://www.youtube.com/watch?v=7rIJNT7dCmE
2538,2539,2492,Epicurious,3800000,False,How To Cut Every Cheese | Method Mastery | Epi...,7579851,178203,10343,,16/04/2021,https://www.youtube.com/watch?v=fTgm36y884c
2539,2540,2492,Rogue Fitness,730000,False,2021 Rogue Invitational - Strongman Competitio...,101267,2591,270,,16/12/2021,https://www.youtube.com/watch?v=kir1The74wk
2540,2541,2492,Flash-224,38100,False,FIA F1 Champions' Press Conference ahead of th...,127573,1845,552,,16/12/2021,https://www.youtube.com/watch?v=yLq366hNo3o
